In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Apr  1 13:00:37 2019

@author: tornado
"""
from tensorflow.keras.layers import TimeDistributed, Activation, Dense, Input, Bidirectional, LSTM, Masking, GaussianNoise
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from CTCModel import CTCModel
import pickle
from tensorflow.keras.preprocessing import sequence
import numpy as np

from read_images import prepareData

#%%
input_data = Input(shape= (144, 64, 32, 1))  #frame, height, width, channels

convolution1 = TimeDistributed(Conv2D(filters=64, kernel_size=(1,1), activation='relu'))(input_data)
bn = TimeDistributed(BatchNormalization())(convolution1)
convolution1 = TimeDistributed(Conv2D(filters=64, kernel_size=(1,1), activation='relu'))(bn)
bn = TimeDistributed(BatchNormalization())(convolution1)
pooling1 = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(convolution1)

convolution2 = TimeDistributed(Conv2D(filters=128, kernel_size=(1,1), activation='relu'))(pooling1)
bn = TimeDistributed(BatchNormalization())(convolution2)
convolution2 = TimeDistributed(Conv2D(filters=128, kernel_size=(1,1), activation='relu'))(convolution2)
bn = TimeDistributed(BatchNormalization())(convolution1)
pooling2 = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(bn)

convolution3 = TimeDistributed(Conv2D(filters=256, kernel_size=(1,1), activation='relu'))(pooling2)
bn = TimeDistributed(BatchNormalization())(convolution3)
convolution3 = TimeDistributed(Conv2D(filters=256, kernel_size=(1,1), activation='relu'))(convolution3)
bn = TimeDistributed(BatchNormalization())(convolution3)
pooling3 = TimeDistributed(MaxPooling2D(pool_size=(2,2)))(bn)

flatten = TimeDistributed(Flatten())(pooling3)

dense = TimeDistributed(Dense(24))(flatten)

blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(dense)
blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)
y_pred = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)

dense = TimeDistributed(Dense(49 + 1, name="dense"))(y_pred)

Model(inputs = input_data, outputs = dense).summary()

network = CTCModel([input_data], [y_pred])
network.compile(Adam(learning_rate=0.0001))


#%%    
x_train, y_train, x_train_len, y_train_len = prepareData(r"E:\Code\Test\TimeDistributed-CRNN\Data\final.csv")

x_test_pad, y_test_pad, x_test_len, y_test_len = prepareData(r"E:\Code\Test\TimeDistributed-CRNN\Data\final.csv")

nb_labels = 49# number of labels (10, this is digits)
batch_size = 5 # size of the batch that are considered
padding_value = 1305 # value for padding input observations
nb_epochs = 1 # number of training epochs
nb_train = 1633
nb_test = 1633
nb_features = 64

# define a recurrent network using CTCModel
#network = create_network(nb_features, nb_labels, padding_value)

# CTC training
network.fit(x=[x_train, y_train, x_train_len, y_train_len], y=np.zeros(nb_train), \
            batch_size=batch_size, epochs=2)

#%%

network.save_model(r"E:\Code\Test\TimeDistributed-CRNN\chomu")
print(r"Saved model to disk")

# Evaluation: loss, label error rate and sequence error rate are requested
eval = network.evaluate(x=[x_test_pad, y_test_pad, x_test_len, y_test_len],\
                        batch_size=batch_size, metrics=['loss', 'ler', 'ser'])


# predict label sequences
pred = network.predict(x =[x_test_pad, x_test_len], batch_size=batch_size, max_value=padding_value)
for i in range(100):  # print the 10 first predictions
    print("Prediction :", [j for j in pred[i] if j!=-1], " -- Label : ", y_train[i]) # 

e:\program_files\conda\envs\test2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
e:\program_files\conda\envs\test2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
e:\program_files\conda\envs\test2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
e:\program_files\conda\envs\test2\lib\site-packages\tensorflow\python\framework

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 144, 64, 32, 1)]  0         
_________________________________________________________________
time_distributed (TimeDistri (None, 144, 64, 32, 64)   128       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 144, 64, 32, 64)   256       
___________________________________________________

Reading file: 92
107 2124
(1270, 64)
Reading file: 93
363 2406
(424, 64)
Reading file: 94
127 2165
(1091, 64)
Reading file: 95
146 1829
(802, 64)
Reading file: 96
131 1976
(965, 64)
Reading file: 97
147 1910
(832, 64)
Reading file: 98
755 1972
(167, 64)
Reading file: 99
209 2232
(683, 64)
Reading file: 100
153 2403
(1005, 64)
Reading file: 101
104 2343
(1442, 64)
Reading file: 102
116 2255
(1244, 64)
Reading file: 103
853 1108
(83, 64)
Reading file: 104
118 2382
(1292, 64)
Reading file: 105
146 2324
(1019, 64)
Reading file: 106
240 2459
(656, 64)
Reading file: 107
130 2410
(1186, 64)
Reading file: 108
144 2434
(1082, 64)
Reading file: 109
153 2384
(997, 64)
Reading file: 110
131 2138
(1045, 64)
Reading file: 111
88 2425
(1764, 64)
Reading file: 112
96 2092
(1395, 64)
Reading file: 113
112 2132
(1218, 64)
Reading file: 114
765 470
(39, 64)
Reading file: 115
82 2334
(1822, 64)
Reading file: 116
111 2172
(1252, 64)
Reading file: 117
115 2096
(1166, 64)
Reading file: 118
405 2258
(357, 64)

145 2203
(972, 64)
Reading file: 319
164 2014
(786, 64)
Reading file: 320
115 1692
(942, 64)
Reading file: 321
143 1672
(748, 64)
Reading file: 322
142 1605
(723, 64)
Reading file: 323
656 1003
(98, 64)
Reading file: 324
147 2139
(931, 64)
Reading file: 325
129 1882
(934, 64)
Reading file: 326
130 1867
(919, 64)
Reading file: 327
125 1878
(962, 64)
Reading file: 328
181 1261
(446, 64)
Reading file: 329
155 2296
(948, 64)
Reading file: 330
140 2166
(990, 64)
Reading file: 331
147 2082
(906, 64)
Reading file: 332
124 1971
(1017, 64)
Reading file: 333
558 812
(93, 64)
Reading file: 334
167 2399
(919, 64)
Reading file: 335
120 2168
(1156, 64)
Reading file: 336
127 2057
(1037, 64)
Reading file: 337
131 612
(299, 64)
Reading file: 338
623 694
(71, 64)
Reading file: 339
112 1881
(1075, 64)
Reading file: 340
129 1895
(940, 64)
Reading file: 341
307 2263
(472, 64)
Reading file: 342
189 2382
(807, 64)
Reading file: 343
156 2047
(840, 64)
Reading file: 344
167 2017
(773, 64)
Reading file: 345
239

120 1392
(742, 64)
Reading file: 540
136 2234
(1051, 64)
Reading file: 541
123 2069
(1077, 64)
Reading file: 542
124 2193
(1132, 64)
Reading file: 543
714 1910
(171, 64)
Reading file: 544
108 2477
(1468, 64)
Reading file: 545
135 2242
(1063, 64)
Reading file: 546
133 2196
(1057, 64)
Reading file: 547
125 2170
(1111, 64)
Reading file: 548
154 2480
(1031, 64)
Reading file: 549
125 2357
(1207, 64)
Reading file: 550
146 2119
(929, 64)
Reading file: 551
858 2107
(157, 64)
Reading file: 552
131 2158
(1054, 64)
Reading file: 553
110 2101
(1222, 64)
Reading file: 554
103 2062
(1281, 64)
Reading file: 555
136 2180
(1026, 64)
Reading file: 556
133 1886
(908, 64)
Reading file: 557
118 1755
(952, 64)
Reading file: 558
122 1845
(968, 64)
Reading file: 559
119 1884
(1013, 64)
Reading file: 560
497 1332
(172, 64)
Reading file: 561
122 1854
(973, 64)
Reading file: 562
111 1821
(1050, 64)
Reading file: 563
117 1792
(980, 64)
Reading file: 564
156 1871
(768, 64)
Reading file: 565
81 1518
(1199, 64)
Read

...

(129, 64)
Reading file: 687
106 2305
(1392, 64)
Reading file: 688
147 2068
(900, 64)
Reading file: 689
164 2051
(800, 64)
Reading file: 690
148 1369
(592, 64)
Reading file: 691
127 2407
(1213, 64)
Reading file: 692
158 2065
(836, 64)
Reading file: 693
119 2367
(1273, 64)
Reading file: 694
142 2138
(964, 64)
Reading file: 695
166 2207
(851, 64)
Reading file: 696
202 2185
(692, 64)
Reading file: 697
104 1523
(937, 64)
Reading file: 698
239 2390
(640, 64)
Reading file: 699
189 2129
(721, 64)
Reading file: 700
743 945
(81, 64)
Reading file: 701
238 2418
(650, 64)
Reading file: 702
507 1555
(196, 64)
Reading file: 703
224 2176
(622, 64)
Reading file: 704
296 2392
(517, 64)
Reading file: 705
112 2080
(1189, 64)
Reading file: 706
98 2167
(1415, 64)
Reading file: 707
866 2467
(182, 64)
Reading file: 708
110 2302
(1339, 64)
Reading file: 709
109 1589
(933, 64)
Reading file: 710
138 2038
(945, 64)
Reading file: 711
147 2099
(914, 64)
Reading file: 712
128 1371
(686, 64)
Reading file: 713
177 243

(954, 64)
Reading file: 910
163 2268
(891, 64)
Reading file: 911
82 783
(611, 64)
Reading file: 912
114 2431
(1365, 64)
Reading file: 913
166 2190
(844, 64)
Reading file: 914
152 2430
(1023, 64)
Reading file: 915
179 2226
(796, 64)
Reading file: 916
211 2244
(681, 64)
Reading file: 917
146 2382
(1044, 64)
Reading file: 918
204 2137
(670, 64)
Reading file: 919
171 2092
(783, 64)
Reading file: 920
171 2026
(758, 64)
Reading file: 921
160 1333
(533, 64)
Reading file: 922
110 2397
(1395, 64)
Reading file: 923
158 2177
(882, 64)
Reading file: 924
183 2267
(793, 64)
Reading file: 925
153 2109
(882, 64)
Reading file: 926
158 2213
(896, 64)
Reading file: 927
719 2278
(203, 64)
Reading file: 928
170 2227
(838, 64)
Reading file: 929
116 1880
(1037, 64)
Reading file: 930
157 2110
(860, 64)
Reading file: 931
154 1990
(827, 64)
Reading file: 932
110 1094
(637, 64)
Reading file: 933
157 2249
(917, 64)
Reading file: 934
874 1523
(112, 64)
Reading file: 935
141 2200
(999, 64)
Reading file: 936
107 776

185 2299
(795, 64)
Reading file: 1134
167 2274
(871, 64)
Reading file: 1135
118 2380
(1291, 64)
Reading file: 1136
98 2190
(1430, 64)
Reading file: 1137
89 2164
(1556, 64)
Reading file: 1138
103 2147
(1334, 64)
Reading file: 1139
726 716
(63, 64)
Reading file: 1140
131 2409
(1177, 64)
Reading file: 1141
101 2072
(1313, 64)
Reading file: 1142
96 2224
(1483, 64)
Reading file: 1143
105 2220
(1353, 64)
Reading file: 1144
102 1082
(679, 64)
Reading file: 1145
187 2363
(809, 64)
Reading file: 1146
157 2087
(851, 64)
Reading file: 1147
151 2065
(875, 64)
Reading file: 1148
138 2070
(960, 64)
Reading file: 1149
560 1963
(224, 64)
Reading file: 1150
163 2311
(907, 64)
Reading file: 1151
109 1993
(1170, 64)
Reading file: 1152
128 2314
(1157, 64)
Reading file: 1153
129 2091
(1037, 64)
Reading file: 1154
92 2382
(1657, 64)
Reading file: 1155
109 2067
(1214, 64)
Reading file: 1156
96 2066
(1377, 64)
Reading file: 1157
957 2122
(142, 64)
Reading file: 1158
125 2401
(1229, 64)
Reading file: 1159
123 

(1134, 64)
Reading file: 1349
91 2241
(1576, 64)
Reading file: 1350
158 2293
(929, 64)
Reading file: 1351
107 2129
(1273, 64)
Reading file: 1352
135 2207
(1046, 64)
Reading file: 1353
869 1638
(121, 64)
Reading file: 1354
125 2386
(1222, 64)
Reading file: 1355
114 2256
(1267, 64)
Reading file: 1356
138 2198
(1019, 64)
Reading file: 1357
348 1346
(248, 64)
Reading file: 1358
192 2359
(786, 64)
Reading file: 1359
239 1990
(533, 64)
Reading file: 1360
705 1876
(170, 64)
Reading file: 1361
172 2342
(871, 64)
Reading file: 1362
152 2295
(966, 64)
Reading file: 1363
153 2204
(922, 64)
Reading file: 1364
125 2251
(1153, 64)
Reading file: 1365
120 2270
(1211, 64)
Reading file: 1366
624 1331
(137, 64)
Reading file: 1367
166 2480
(956, 64)
Reading file: 1368
178 2199
(791, 64)
Reading file: 1369
105 2249
(1371, 64)
Reading file: 1370
115 2142
(1192, 64)
Reading file: 1371
145 2274
(1004, 64)
Reading file: 1372
117 2259
(1236, 64)
Reading file: 1373
135 2359
(1118, 64)
Reading file: 1374
119 2178

(145, 64)
Reading file: 1566
140 2177
(995, 64)
Reading file: 1567
176 2003
(728, 64)
Reading file: 1568
167 1968
(754, 64)
Reading file: 1569
159 1937
(780, 64)
Reading file: 1570
152 1449
(610, 64)
Reading file: 1571
199 2467
(793, 64)
Reading file: 1572
144 2203
(979, 64)
Reading file: 1573
147 2219
(966, 64)
Reading file: 1574
135 2173
(1030, 64)
Reading file: 1575
662 2349
(227, 64)
Reading file: 1576
187 2115
(724, 64)
Reading file: 1577
145 1650
(728, 64)
Reading file: 1578
143 2375
(1063, 64)
Reading file: 1579
129 2049
(1017, 64)
Reading file: 1580
135 2201
(1043, 64)
Reading file: 1581
166 2120
(817, 64)
Reading file: 1582
98 2300
(1502, 64)
Reading file: 1583
137 2163
(1010, 64)
Reading file: 1584
155 2105
(869, 64)
Reading file: 1585
162 1781
(704, 64)
Reading file: 1586
107 1921
(1149, 64)
Reading file: 1587
927 1242
(86, 64)
Reading file: 1588
84 1394
(1062, 64)
Reading file: 1589
102 2056
(1290, 64)
Reading file: 1590
132 2166
(1050, 64)
Reading file: 1591
124 2336
(1206